# Отчет по лабораторной работе #5
### Выполнила: Казакова Анна

Исследование проводилось на наборе данных [Dogs vs. Cats](www.kaggle.com/c/dogs-vs-cats)

#### Описание выборки:
* Содержит изображения в формате JPEG с низким разрешением
* Содержит 25 000 изображений кошек и собак (по 12 500 для каждого класса)

Будем решать задачу с использованием сверточной нейронной сети т.к. именно этот подход показывает себя лучше всего при классификации изображений. Для этого нам понадобятся следующие библиотеки и их объекты

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf

from keras import layers, regularizers
from keras import models
from keras import optimizers
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.backend.tensorflow_backend import set_session

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

Опишем функции для метрик precision и recall

In [2]:
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

Сконструируем нашу модель сверточной нейронной сети. На этапе компиляции, как обычно, используем оптимизатор RMSprop. Так как сеть
заканчивается единственным признаком, используем функцию потерь binary_crossentropy

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc', recall, precision])
model.summary()

Перед передачей в сеть данные должны быть преобразованы
в тензоры с вещественными числами. Прочитаем файлы, декодируем в PNG, преобразуем в тензоры и отмаштабируем из диапазона 0..255 в 0..1 использую ImageDataGenerator библиотеки Keras. После этого передадим получившийся generator на вход функции fit_generator. Для получения 2000 образцов модель извлечет 100 пакетов, т.к. пакеты содержат по 20 образцов. Стоит отметить, что здесь мы уже провели расширение данных после сильного переобучения модели, используя rotation_range, width/height_shift, shear_range и zoom_range, horizontal_flip, fill_mode. Сохраним модель после обучения

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2,
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=20, class_mode='binary')
validation_generator = test_datagen.flow_from_directory(validation_dir, target_size=(224, 224), batch_size=20, class_mode='binary')
history = model.fit_generator(train_generator, steps_per_epoch=400, epochs=30, validation_data=validation_generator, validation_steps=100)
model.save('cats-and-dogs.h5')

Выведем значения получившихся метрик после обучения

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
print('accuracy, val_accuracy:', acc, val_acc)

recall = history.history['recall']
val_recall = history.history['val_recall']
print('recall, val_recall:', recall, val_recall)

precision = history.history['precision']
val_precision = history.history['val_precision']
print('precision, val_precision:', precision, val_precision)

loss = history.history['loss']
val_loss = history.history['val_loss']
print('loss, val_loss:', loss, val_loss)

Теперь создадим графики изменения таких метрик как: accuracy, recall, precision, loss

In [ ]:
epochs = range(len(acc))
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.savefig("accuracy.png")

plt.plot(epochs, recall, 'bo', label='Training recall')
plt.plot(epochs, val_recall, 'b', label='Validation recall')
plt.title('Training and validation recall')
plt.legend()
plt.savefig("recall.png")

plt.plot(epochs, precision, 'bo', label='Training precision')
plt.plot(epochs, val_precision, 'b', label='Validation precision')
plt.title('Training and validation precision')
plt.legend()
plt.savefig("precision.png")

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.savefig("loss.png")

Выводы:
 Мы практически избавились от переобучения с помощью расширения данных и прореживания, настройки сверточной сети. Мы достигли точности ~82%, однако такой показатель может быть улучшен последующей детальной настройкой сети или манипуляциями с данными. Также мы убедились, что сверточные сети являются лучшим типом моделей для обучения распознавания образов. Опять же, когда мы имеем ограниченый набор данных, переобучение становится основной и даже порой единственной проблемой. В этом случае помогла работа с данными, преобразования над ними путем их расширения